<a href="https://colab.research.google.com/github/fjmsouza/ML2DL/blob/main/code_example_two_paths_v2_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
monitor='val_accuracy'
epochs=30
batch_size=16
input_shape=(128, 128, 3) # please resize it to (224,224,3) if you have enough RAM
Verbose=True

This source code requires a **HIGH RAM** machine.

You might need to install this on your system:

apt-get install python3-opencv git

In [ ]:
import os

if not os.path.isdir('k'):
  !git clone -b development16 https://github.com/joaopauloschuler/k-neural-api.git k
else:
  !cd k && git pull

!cd k && pip install .

Cloning into 'k'...
remote: Enumerating objects: 1796, done.
remote: Counting objects: 100% (208/208), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 1796 (delta 135), reused 123 (delta 63), pack-reused 1588
Receiving objects: 100% (1796/1796), 15.66 MiB | 13.38 MiB/s, done.
Resolving deltas: 100% (1248/1248), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/k
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for cai: filename=cai-0.1.6-py3-none-any.whl size=60126 sha256=14affffd8baec208ee4114d9502717cd4e0f34e2d6a68e8fb6436cd62f3ff9

In [ ]:
import sys
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)

import skimage
print('skimage version',  skimage.__version__)
import tensorflow as tf
print(tf.__version__)
import sys
import cai
import cai.datasets
import cai.models

Python version
3.7.15 (default, Oct 12 2022, 19:14:55) 
[GCC 7.5.0]
Version info.
sys.version_info(major=3, minor=7, micro=15, releaselevel='final', serial=0)
skimage version 0.18.3
2.9.2


In [ ]:
url_zip_file="https://data.mendeley.com/public-files/datasets/tywbtsjrjv/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/file_downloaded"
local_zip_file="plant_leaf.zip"
expected_folder_name="plant_leaf"
cai.datasets.download_zip_and_extract(
    url_zip_file=url_zip_file, local_zip_file=local_zip_file, 
    expected_folder_name=expected_folder_name, Verbose=Verbose)

Downloading:  https://data.mendeley.com/public-files/datasets/tywbtsjrjv/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/file_downloaded  to  plant_leaf.zip
Decompressing into:  plant_leaf


In [ ]:
import tensorflow
import multiprocessing

In [ ]:
!rm -r plant_leaf/Plant_leave_diseases_dataset_without_augmentation/Background_without_leaves -R
data_dir = "plant_leaf/Plant_leave_diseases_dataset_without_augmentation/"
print(os.listdir(data_dir))
print(len(os.listdir(data_dir)))

rm: cannot remove 'plant_leaf/Plant_leave_diseases_dataset_without_augmentation/Background_without_leaves': No such file or directory
['Tomato___Leaf_Mold', 'Corn___Northern_Leaf_Blight', 'Corn___Common_rust', 'Strawberry___Leaf_scorch', 'Blueberry___healthy', 'Tomato___Late_blight', 'Strawberry___healthy', 'Grape___Esca_(Black_Measles)', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___healthy', 'Potato___Early_blight', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Peach___Bacterial_spot', 'Tomato___Tomato_mosaic_virus', 'Pepper,_bell___healthy', 'Tomato___Septoria_leaf_spot', 'Cherry___Powdery_mildew', 'Apple___Black_rot', 'Pepper,_bell___Bacterial_spot', 'Apple___Apple_scab', 'Apple___Cedar_apple_rust', 'Soybean___healthy', 'Tomato___Bacterial_spot', 'Potato___healthy', 'Cherry___healthy', 'Tomato___healthy', 'Grape___Black_rot', 'Apple___healthy', 'Raspberry___healthy', 'Grape___healthy'

In [ ]:
train_x, val_x, test_x, train_y, val_y, test_y, classweight, classes = cai.datasets.load_images_from_folders(seed=7, root_dir=data_dir, lab=True, 
  verbose=Verbose, bipolar=False, base_model_name='plant_leaf',
  training_size=0.6, validation_size=0.2, test_size=0.2,
  target_size=(input_shape[0],input_shape[1]), 
  has_training=True, has_validation=True, has_testing=True, 
  smart_resize=True)

Loading  38  classes.
smart resize is enabled.
loading train images
train shape is: (32571, 128, 128, 3)
loading validation images
validation shape is: (10858, 128, 128, 3)
loading test images
test shape is: (10876, 128, 128, 3)
Channel  0  min: 0.0  max: 1.0
Channel  1  min: 0.19593212  max: 0.88375795
Channel  2  min: 0.2551091  max: 0.9470912
Loaded.


In [ ]:
print(train_x.shape,val_x.shape,test_x.shape)
print(train_y.shape,val_y.shape,test_y.shape)

(32571, 128, 128, 3) (10858, 128, 128, 3) (10876, 128, 128, 3)
(32571, 38) (10858, 38) (10876, 38)


In [ ]:
for two_paths_second_block in [False]:
  for l_ratio in [0.2]:
    basefilename = 'two-path-inception-v2.8-'+str(two_paths_second_block)+'-'+str(l_ratio)
    print('Running: '+basefilename)
    model = cai.models.compiled_two_path_inception_v3(
      input_shape=input_shape,
      classes=38, 
      two_paths_first_block=True,
      two_paths_second_block=two_paths_second_block,
      l_ratio=l_ratio,
      ab_ratio=(1-l_ratio),
      max_mix_idx=5, 
      model_name='two_path_inception_v3'
      )    
    monitor='val_accuracy'
    best_result_file_name = basefilename+'-best_result.hdf5'
    save_best = tensorflow.keras.callbacks.ModelCheckpoint(
      filepath=best_result_file_name, 
      monitor=monitor, 
      verbose=1, 
      save_best_only=True, 
      save_weights_only=False, 
      mode='max', 
      period=1)
    history = model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size,
      validation_data=(val_x,val_y),
      callbacks=[save_best],class_weight=classweight,
      workers=multiprocessing.cpu_count())
    print('Testing Last Model: '+basefilename)
    evaluated = model.evaluate(test_x,test_y)
    for metric, name in zip(evaluated,["loss","acc","top 5 acc"]):
      print(name,metric)
    print('Best Model Results: '+basefilename)
    model = tensorflow.keras.models.load_model(best_result_file_name, custom_objects={'CopyChannels': cai.layers.CopyChannels})
    evaluated = model.evaluate(test_x,test_y)
    cai.models.save_model(model, basefilename)
    for metric, name in zip(evaluated,["loss","acc","top 5 acc"]):
      print(name,metric)
    print('Finished: '+basefilename)

Running: two-path-inception-v2.8-False-0.2


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/30
2036/2036 [==============================] - ETA: 0s - loss: 1.4232 - accuracy: 0.5826
Epoch 1: val_accuracy improved from -inf to 0.67839, saving model to two-path-inception-v2.8-False-0.2-best_result.hdf5
2036/2036 [==============================] - 1335s 654ms/step - loss: 1.4232 - accuracy: 0.5826 - val_loss: 1.1257 - val_accuracy: 0.6784
Epoch 2/30
2036/2036 [==============================] - ETA: 0s - loss: 0.5310 - accuracy: 0.8194
Epoch 2: val_accuracy did not improve from 0.67839
2036/2036 [==============================] - 1387s 681ms/step - loss: 0.5310 - accuracy: 0.8194 - val_loss: 2.3552 - val_accuracy: 0.5136
Epoch 3/30
2036/2036 [==============================] - ETA: 0s - loss: 0.3573 - accuracy: 0.8742
Epoch 3: val_accuracy improved from 0.67839 to 0.90376, saving model to two-path-inception-v2.8-False-0.2-best_result.hdf5
2036/2036 [==============================] - 1395s 685ms/step - loss: 0.3573 - accuracy: 0.8742 - val_loss: 0.2956 - val_accuracy: 0.903

KeyboardInterrupt: ignored